In [5]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from keras.preprocessing.sequence import pad_sequences

In [6]:
# Load the dataset
df = pd.read_csv("input/spamorham.csv")

# Preprocessing the data

In [40]:
print(df.shape)
df.dropna()
print(df.shape)

(20348, 2)
(20348, 2)


In [35]:

# "text" column contains the message text and "text_type" contains the labels (spam or not)
X = df['text']
y = df['text_type']

# Convert labels to numerical values
le = LabelEncoder()
y = le.fit_transform(y)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Tokenize the text data
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(X_train)

# Convert text to sequences
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Pad sequences to make them uniform length
max_sequence_length = 100  # Choose the maximum length of sequences
X_train_pad = tf.keras.preprocessing.sequence.pad_sequences(X_train_seq, maxlen=max_sequence_length)
X_test_pad = tf.keras.preprocessing.sequence.pad_sequences(X_test_seq, maxlen=max_sequence_length)

# Create model

In [41]:
# Define the model architecture
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=64, input_length=max_sequence_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train_pad, y_train, epochs=10, batch_size=32, validation_data=(X_test_pad, y_test))

Epoch 1/10


w:\__Projects__\Deep-Learning-Experiments\.venv\Lib\site-packages\keras\src\layers\core\embedding.py:86: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


509/509 ━━━━━━━━━━━━━━━━━━━━ 13s 23ms/step - accuracy: 0.7399 - loss: 0.5066 - val_accuracy: 0.9081 - val_loss: 0.2115
Epoch 2/10
509/509 ━━━━━━━━━━━━━━━━━━━━ 11s 22ms/step - accuracy: 0.9468 - loss: 0.1582 - val_accuracy: 0.9437 - val_loss: 0.1397
Epoch 3/10
509/509 ━━━━━━━━━━━━━━━━━━━━ 11s 22ms/step - accuracy: 0.9695 - loss: 0.0930 - val_accuracy: 0.9619 - val_loss: 0.1155
Epoch 4/10
509/509 ━━━━━━━━━━━━━━━━━━━━ 11s 21ms/step - accuracy: 0.9790 - loss: 0.0660 - val_accuracy: 0.9526 - val_loss: 0.1345
Epoch 5/10
509/509 ━━━━━━━━━━━━━━━━━━━━ 11s 21ms/step - accuracy: 0.9833 - loss: 0.0503 - val_accuracy: 0.9376 - val_loss: 0.1883
Epoch 6/10
509/509 ━━━━━━━━━━━━━━━━━━━━ 11s 22ms/step - accuracy: 0.9861 - loss: 0.0462 - val_accuracy: 0.9523 - val_loss: 0.1472
Epoch 7/10
509/509 ━━━━━━━━━━━━━━━━━━━━ 11s 21ms/step - accuracy: 0.9879 - loss: 0.0362 - val_accuracy: 0.9496 - val_loss: 0.1572
Epoch 8/10
509/509 ━━━━━━━━━━━━━━━━━━━━ 11s 21ms/step - accuracy: 0.9872 - loss: 0.0388 - val_accurac

## Save & Load the model

In [43]:
#Save the model
model.save("output/spam_or_ham_model.keras")

In [44]:
#Load the model
model = tf.keras.models.load_model("output/spam_or_ham_model.keras")

w:\__Projects__\Deep-Learning-Experiments\.venv\Lib\site-packages\keras\src\saving\saving_lib.py:418: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 7 variables whereas the saved optimizer has 12 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


# Evaluate the model

Evaluation of the model is done on the test set, which is not available to us. Therefore, we will split the training data into training and validation sets. We will use the validation set to tune the hyperparameters of the model.

In [46]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test_pad, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

# Further evaluation metrics
y_pred = model.predict(X_test_pad)
y_pred = (y_pred > 0.5) # Convert the probabilities to binary predictions

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(cm)

##classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))

128/128 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9625 - loss: 0.1387
Test Loss: 0.15683908760547638
Test Accuracy: 0.9582309722900391
128/128 ━━━━━━━━━━━━━━━━━━━━ 0s 752us/step
Confusion Matrix:
[[2856   57]
 [ 113 1044]]
Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.98      0.97      2913
           1       0.95      0.90      0.92      1157

    accuracy                           0.96      4070
   macro avg       0.96      0.94      0.95      4070
weighted avg       0.96      0.96      0.96      4070



Evaluate with user defined test sets

In [26]:
my_text="Congratulations! You have been selected as a winner. Text WON to 44255 to claim your prize."


In [27]:
# Example of classifying a new text_message
#new_text_message = ["Your new text_message text goes here"]
new_text_message = [my_text]
new_text_message_seq = tokenizer.texts_to_sequences(new_text_message)
new_text_message_pad = pad_sequences(new_text_message_seq, maxlen=max_sequence_length)
prediction = model.predict(new_text_message_pad)

#Print the prediction (0 = ham, 1 = spam) and print the accuracy of the model
if prediction[0][0] >= 0.5:
    print("The text message is spam.")
    #print accuracy of the model
    print("At accuracy: ", accuracy)
    #print how sure it is that the text message is spam
    print("The model is ", prediction[0][0]*100, "% sure that the text message is spam.")
    
else:
    print("The text message is ham.")
    #print accuracy of the model
    print("At accuracy: ", accuracy)
    #print how sure it is that the text message is ham
    print("The model is ", (1-prediction[0][0])*100, "% sure that the text message is not spam (ham).")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
The text message is spam.
At accuracy:  0.954054057598114
The model is  90.2423620223999 % sure that the text message is spam.
